In [ ]:
import mysql.connector
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import folium

import warnings
warnings.simplefilter('ignore')

In [ ]:
with open('user.txt', 'r') as file:
    user, pwd = file.read().split('\n')

mydb = mysql.connector.connect(host="localhost", user=user, password=pwd, database='DMARC')
mycursor = mydb.cursor()
database = pd.read_sql('SELECT * FROM RUA', mydb)

In [ ]:
sum(database['count'])

In [ ]:
spf = database['spf_aligned'].value_counts().rename_axis('labels').reset_index(name='count')
if len(spf) == 2:
    spf_aligned = [spf['count'][0], spf['count'][1]]
elif len(spf) == 1 and spf['labels'][0] == '0':
    spf_aligned = [spf['count'][0], 0]
elif len(spf) == 1 and spf['labels'][0] == '1':
    spf_aligned = [0, spf['count'][0]]

dkim = database['dkim_aligned'].value_counts().rename_axis('labels').reset_index(name='count')
if len(dkim) == 2:
    dkim_aligned = [dkim['count'][0], dkim['count'][1]]
elif len(dkim) == 1 and dkim['labels'][0] == '0':
    dkim_aligned = [dkim['count'][0], 0]
elif len(dkim) == 1 and dkim['labels'][0] == '1':
    dkim_aligned = [0, dkim['count'][0]]

dmarc = database['dmarc_aligned'].value_counts().rename_axis('labels').reset_index(name='count')
if len(dmarc) == 2:
    dmarc_aligned = [dmarc['count'][0], dmarc['count'][1]]
elif len(dmarc) == 1 and dmarc['labels'][0] == '0':
    dmarc_aligned = [dmarc['count'][0], 0]
elif len(dmarc) == 1 and dmarc['labels'][0] == '1':
    dmarc_aligned = [0, dmarc['count'][0]]

In [ ]:
fig = make_subplots(rows=1, cols=3, specs=[[{"type": "pie"}, {"type": "pie"}, {"type": "pie"}]], 
                    subplot_titles=('SPF Alignment', 'DKIM Alignment', 'DMARC Alignment'))

fig.add_trace(go.Pie(values=spf_aligned, labels=['False','True'], domain=dict(x=[0, 0.5])), row=1, col=1)
fig.add_trace(go.Pie(values=dkim_aligned, labels=['False','True'], domain=dict(x=[0, 0.5])), row=1, col=2)
fig.add_trace(go.Pie(values=dmarc_aligned, labels=['False','True'], domain=dict(x=[0, 0.5])), row=1, col=3)
fig.show()

In [ ]:
fig = px.pie(database, names='spf_aligned', color='spf_aligned', color_discrete_map={'0':'gold', '1':'green'}, title='SPF Alignment')
fig.update_layout(autosize=False, width=400, height=400)
fig.show()

In [ ]:
fig = px.pie(database, names='dkim_aligned', color='dkim_aligned', color_discrete_map={'0':'gold', '1':'green'}, title='DKIM Alignment')
fig.update_layout(autosize=False, width=400, height=400)
fig.show()

In [ ]:
fig = px.pie(database, names='dmarc_aligned', color='dmarc_aligned', color_discrete_map={'0':'crimson', '1':'green'}, title='DMARC Alignment')
fig.update_layout(autosize=False, width=400, height=400)
fig.show()

In [ ]:
database['day'] = database['begin_date'].str.slice(0, 10)
database['spf_0_count'] = np.where(database['spf_aligned'] == '0', database['count'], 0)
database['spf_1_count'] = np.where(database['spf_aligned'] == '1', database['count'], 0)
database['dkim_0_count'] = np.where(database['dkim_aligned'] == '0', database['count'], 0)
database['dkim_1_count'] = np.where(database['dkim_aligned'] == '1', database['count'], 0)
database['dmarc_0_count'] = np.where(database['dmarc_aligned'] == '0', database['count'], 0)
database['dmarc_1_count'] = np.where(database['dmarc_aligned'] == '1', database['count'], 0)

In [ ]:
db_day = database[['spf_0_count','spf_1_count','dkim_0_count','dkim_1_count','dmarc_0_count','dmarc_1_count']].groupby(by=database['day']).sum()
db_day.reset_index(inplace=True)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=db_day['day'], y=db_day['spf_0_count'], line=dict(color='gold', width=2), name='fail'))
fig.add_trace(go.Scatter(x=db_day['day'], y=db_day['spf_1_count'], line=dict(color='green', width=2), name='pass'))
fig.update_layout(title='SPF Results Over Time')
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=db_day['day'], y=db_day['dkim_0_count'], line=dict(color='gold', width=2), name='fail'))
fig.add_trace(go.Scatter(x=db_day['day'], y=db_day['dkim_1_count'], line=dict(color='green', width=2), name='pass'))
fig.update_layout(title='DKIM Results Over Time')
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=db_day['day'], y=db_day['dmarc_0_count'], line=dict(color='crimson', width=2), name='fail'))
fig.add_trace(go.Scatter(x=db_day['day'], y=db_day['dmarc_1_count'], line=dict(color='green', width=2), name='pass'))
fig.update_layout(title='DMARC Results Over Time')
fig.show()

In [ ]:
countries = pd.read_csv('data/country-coord.csv')
countries = countries[['Country', 'Alpha-2 code','Latitude (average)', 'Longitude (average)']]
countries.rename(columns={'Country':'name', 'Alpha-2 code':'source_country', 'Latitude (average)':'lat', 'Longitude (average)':'long'}, inplace=True)

In [ ]:
db_iso = database[['count']].groupby(by=database['source_country']).sum()
db_iso.reset_index(inplace=True)
db_iso = db_iso.merge(countries, on='source_country')
db_iso['color'] = np.where(db_iso['count'] >= 500, 'crimson', np.where(db_iso['count'] >= 10, 'gold', 'green'))

In [ ]:
m = folium.Map()

db_iso.apply(lambda row:folium.CircleMarker(
    location=[row['lat'], row['long']], radius=row['count'], popup=row['name'] + '\n' + str(row['count']), fill=True, color=row['color'], fill_color=row['color']
).add_to(m), axis=1)

m

In [ ]:
db_ip = database[['source_reverse_dns', 'source_base_domain', 'count']].groupby(by=database['source_ip_address']).aggregate({'source_reverse_dns':'first',
                                                                                                                             'source_base_domain': 'first', 'count':'sum'})
db_ip.sort_values(by='count', ascending=False)